In [ ]:
import os
import glob
import gdown
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression

In [ ]:
def download_weights():
    model_file=[
        'face_detection.weights',
        'face_detection.cfg'
    ]
    
    gdrive_url=[
        'https://drive.google.com/uc?id=1nYY0GbZqhssZvkH14WGo9vJG7y9pfMtV',
        'https://drive.google.com/uc?id=1IyGoLxm2VPmtt8NMhZp8KueilzaRwdfg'
    ]
    
    cwd=os.getcwd() 
    if 'weights' in os.listdir(cwd):
        for i in range(len(model_file)):
            if model_file[i] in os.listdir(os.path.join(cwd, 'weights')):
                print(model_file[i] + ':: status : file already exists')
            else:
                gdown.download(gdrive_url[i],os.path.join(cwd, 'weights', model_file[i]), quiet=False)
    else:
        os.makedirs(os.path.join(cwd,'weights'))
        for i in range(len(model_file)):
            gdown.download(gdrive_url[i], os.path.join(cwd, 'weights', model_file[i]), quiet=False)  

In [ ]:
# download the necessary weights for YOLO-Face
download_weights()

## YOLOFace with FGSM

In [ ]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device, model = models.load_model('./weights/face_detection.cfg', "./weights/face_detection.weights")

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

epsilons = [0, .05]
use_cuda=True

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
FOLDER_PATH = os.path.join(os.getcwd(), 'images')
print(FOLDER_PATH)

def test( model, device, epsilon ):

    # Loop over all examples in test set
    for path in glob.glob(os.path.join(FOLDER_PATH, '*.jpg')):
        print(path)
#         print(torch.min(data), torch.max(data))
#         print('Input')
#         print(data.shape)
#         plt.imshow(np.transpose(data.squeeze(0).numpy(), (1, 2, 0)))
#         plt.show()
        
        data = cv2.imread(path)
        print(data.shape)
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    
        data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0)
    
        data = data.to(device)
        
        print('Input')
        plt.imshow(np.transpose(data.squeeze().detach().cpu().numpy(), (1, 2, 0)))
        plt.show()
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        
        # Forward pass the data through the model
        output = model(data)
#         print('Model Output')
#         print(output)
#         print(output.shape)
    
        
        nms, nms_output = non_max_suppression(output, 0.5, 0.5)
        print('NMS')
        print(nms)
        print(nms_output)
        
        for face_index, face_row in enumerate(nms_output[0]): #nms_output[0] because the code/model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time
            print('Face ', face_index)
            print(face_row)

            # Calculate the loss
            #TODO: determine what should be the ground truth
            loss = F.binary_cross_entropy(face_row[5:], torch.tensor([0., 0.]))
            
            #x1 y1 lower left
            #x2 y2 upper right
            x1 = int(np.floor((face_row[0] - face_row[2] / 2).detach().cpu().numpy()))
            y1 = int(np.floor((face_row[1] - face_row[3] / 2).detach().cpu().numpy()))
            x2 = int(np.ceil((face_row[0] + face_row[2] / 2).detach().cpu().numpy()))
            y2 = int(np.ceil((face_row[1] + face_row[3] / 2).detach().cpu().numpy()))
            
            print('Cropped')
            print(x1, y1, x2, y2)
            cropped_image = data[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
            plt.imshow(np.transpose(cropped_image.squeeze().detach().cpu().numpy(), (1, 2, 0)))
            plt.show()
            
            #Resize
            input_cropped_image = np.transpose(cropped_image.squeeze().detach().cpu().numpy(), (1, 2, 0)) #reshape the image to (w/h, h/w, channel)

            print('Resized')
            cropped_resized_image = np.transpose(transforms.Compose([DEFAULT_TRANSFORMS,Resize(128)])((input_cropped_image, np.zeros((1, 5))))[0], (1, 2, 0))
            plt.imshow(cropped_resized_image)
            plt.show()
            
            # Zero all existing gradients
            model.zero_grad()

            # Calculate gradients of model in backward pass
            loss.backward(retain_graph=True) #TODO: check if this is correct

            # Collect datagrad
            data_grad = data.grad.data
            print('Gradient')
            print(data_grad.shape)      
            plt.imshow(np.transpose(np.clip(data_grad.squeeze(0).numpy(), 0, 1), (1, 2, 0)))
            plt.show()

            # Call FGSM Attack
            perturbed_data = fgsm_attack(data, epsilon, data_grad)

            print(torch.min(perturbed_data), torch.max(perturbed_data))
            plt.imshow(np.transpose(perturbed_data.squeeze().detach().cpu().numpy(), (1, 2, 0)))
            plt.show()

In [ ]:
# loops for different epsilon
for eps in epsilons:
    print('Epsilon', eps)
    test(model, device, eps)